In [4]:

import os
import requests
import numpy as np
# Cấu hình Qdrant
QDRANT_API_URL = os.getenv(
    "QDRANT_API_URL",
    "https://fcbf96b5-0f95-47b1-b088-dd1eba2a2758.us-east4-0.gcp.cloud.qdrant.io:6333",
)
QDRANT_API_KEY = os.getenv(
    "QDRANT_API_KEY", "WbQ_8KeZKchBfQ-atnt5zfbkIShw6slMNvF0PK8qIOEIgaYqTyZLmw"
)
QDRANT_COLLECTION = os.getenv("QDRANT_COLLECTION", "cmc_final_db")


def test_qdrant_search():
    """
    Gửi một truy vấn đơn giản tới Qdrant để kiểm tra kết nối và response.
    """
    url = f"{QDRANT_API_URL}/collections/{QDRANT_COLLECTION}/points/search"
    headers = {
        "Authorization": f"Bearer {QDRANT_API_KEY}",
        "Content-Type": "application/json",
    }

    # Tạo một vector ngẫu nhiên kích thước 768
    query_vector = list(np.random.rand(768))  # Hoặc vector thực tế từ pipeline của bạn
    payload = {
        "vector": query_vector,
        "limit": 5,  # Số lượng kết quả muốn lấy
    }

    print("Testing Qdrant search...")
    try:
        response = requests.post(url, json=payload, headers=headers)
        response.raise_for_status()
        print(f"Response Status: {response.status_code}")
        print("Response Data:", response.json())
    except requests.exceptions.HTTPError as e:
        print("HTTP Error:", e.response.status_code, e.response.text)
    except requests.exceptions.RequestException as e:
        print("Error occurred:", str(e))


if __name__ == "__main__":
    test_qdrant_search()


Testing Qdrant search...
Response Status: 200
Response Data: {'result': [{'id': '7c9b8dbf-e631-429d-985d-b443b04a9dad', 'version': 0, 'score': 0.056992024}, {'id': 'a3fc7706-2812-4495-838e-18cbcd5bba89', 'version': 0, 'score': 0.04741749}, {'id': '6a20096d-8342-4b6f-8860-6ac2bc0ad306', 'version': 1, 'score': 0.04589819}, {'id': 'e28045df-f779-462f-a4e7-2d5a7ca66166', 'version': 1, 'score': 0.045791358}, {'id': 'a5693358-ca3e-4d1d-9d4c-b383348a989d', 'version': 2, 'score': 0.04545533}], 'status': 'ok', 'time': 0.001576648}


## Add thêm Embedding Query

In [1]:
import os
import requests
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Qdrant configuration
QDRANT_API_URL = os.getenv("QDRANT_API_URL")
QDRANT_API_KEY = os.getenv("QDRANT_API_KEY")  # Thay bằng API key của bạn
QDRANT_COLLECTION = os.getenv("QDRANT_COLLECTION")

# OpenAI embedding configuration (nếu dùng OpenAI)
OPENAI_API_URL = os.getenv("OPENAI_API_URL")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")  # Thay bằng API key OpenAI của bạn
MODEL_NAME = os.getenv("MODEL_NAME")  # Model embedding từ OpenAI


# Function to generate embedding using OpenAI
def generate_embedding(query_text):
    headers = {
        "Authorization": f"Bearer {OPENAI_API_KEY}",
        "Content-Type": "application/json",
    }
    payload = {
        "input": query_text,
        "model": MODEL_NAME,
    }

    print("Generating embedding for query...")
    response = requests.post(OPENAI_API_URL, json=payload, headers=headers)
    response.raise_for_status()
    data = response.json()
    return data["data"][0]["embedding"]

# Function to search in Qdrant
def search_qdrant(embedding_vector):
    url = f"{QDRANT_API_URL}/collections/{QDRANT_COLLECTION}/points/search"
    headers = {
        "Authorization": f"Bearer {QDRANT_API_KEY}",
        "Content-Type": "application/json",
    }
    payload = {
        "vector": embedding_vector,
        "limit": 5,  # Số lượng kết quả muốn lấy
    }

    print("Testing Qdrant search...")
    response = requests.post(url, json=payload, headers=headers)
    response.raise_for_status()
    return response.json()

# Main flow
def main():
    # Query text
    query_text = "Find information about deep learning and AI models."

    try:
        # Step 1: Generate embedding
        embedding_vector = generate_embedding(query_text)
        print(f"Generated embedding: {embedding_vector[:5]}...")  # In 5 giá trị đầu

        # Step 2: Search in Qdrant
        search_results = search_qdrant(embedding_vector)
        print("Search Results:", search_results)

    except requests.exceptions.HTTPError as e:
        print("HTTP Error:", e.response.status_code, e.response.text)
    except requests.exceptions.RequestException as e:
        print("Error occurred:", str(e))

# Run the main function
if __name__ == "__main__":
    main()


Generating embedding for query...
HTTP Error: 401 {
    "error": {
        "message": "Incorrect API key provided: YOUR_OPE*******_KEY. You can find your API key at https://platform.openai.com/account/api-keys.",
        "type": "invalid_request_error",
        "param": null,
        "code": "invalid_api_key"
    }
}

